# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as images, videos, music, and text, which can be used for marketing campaigns, social media, and advertising.
2. **Personalized Recommendations**: Generative AI algorithms can analyze customer data and behavior to provide personalized product recommendations, enhancing the user experience and driving sales.
3. **Chatbots and Virtual Assistants**: Generative AI-powered chatbots and virtual assistants can understand natural language and respond accordingly, providing 24/7 customer support and improving customer satisfaction.
4. **Automated Customer Service**: Generative AI can generate automated responses to common customer inquiries, reducing the need for human intervention and improving response times.
5. **Design and Prototyping**: Generative AI can assist designers in creating new designs and prototypes by generating mul

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [12]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources on content creation.
2. **Product Design and Development**: Generative AI can be used to design and develop new products, such as fashion items, electronics, or architectural designs.
3. **Marketing Automation**: AI-powered marketing tools can generate personalized marketing messages, emails, and ads based on customer behavior and preferences.
4. **Image and Video Generation**: AI can generate high-quality images and videos for use in advertising, product showcases, and other marketing materials.
5. **Data Augmentation**: Generative AI can be used to augment existing datasets by generating new data points that mimic the patterns found in the original data.
6. **Language Translation**: AI-powered translation tools can ge

## Alternative approach - using OpenAI python library to connect to Ollama

In [1]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

NameError: name 'MODEL' is not defined

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [9]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [18]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
    
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}]
    
    
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [19]:
display_summary("https://edwarddonner.com")

**Summary**
================

* The website is about Edward Donner, a co-founder of Nebula.io and former founder of AI startup untapt.
* He works with artificial intelligence (AI) and language models (LLMs).
* Ed is an enthusiast for DJing, amateur electronic music production, and Hacker News.
* The website includes news/announcements:
	+ December 21, 2024: "Welcome, SuperDataScientists!"
	+ November 13, 2024: Mastering AI and LLM Engineering – Resources
	+ October 16, 2024: From Software Engineer to AI Data Scientist – resources
	+ August 6, 2024: Outsmart LLM Arena – a battle of diplomacy and deviousness

**Key Information**
------------------

* Ed's role as co-founder/CTO at Nebula.io and founder/CEO at untapt.
* Focus on applying AI to help people discover their potential.
* Lattice matchings model patented by Nebula.io.
* Various newsletters subscribed.